In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 13}
----------
[[13567  5793]
 [  997 18329]]
----------
              precision    recall  f1-score   support

           0       0.93      0.70      0.80     19360
           1       0.76      0.95      0.84     19326

    accuracy                           0.82     38686
   macro avg       0.85      0.82      0.82     38686
weighted avg       0.85      0.82      0.82     38686

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.26927996, 0.33211219, 0.38147986, 0.47323382, 0.47522819,
        0.57595873, 0.63430417, 0.64577365, 0.64926434, 0.71608424,
        0.78140926, 0.77841949, 0.77492666, 0.8337698 , 1.01678002,
        0.47871935, 0.54204941, 0.63530099, 0.69015419, 0.70860445,
        0.81332481, 0.86119664, 0.97987998, 0.93400276, 1.06415546,
        1.19829547, 1.27060223, 1.28306949, 1.21175373, 1.14545619]),
 'std_fit_time': array([0.00199389, 0.020944  , 0.02543151, 0.03939283, 0.00548494,
        0.01545978, 0.00997412, 0.04138851, 0.00099659, 0.03690171,
        0.01944757, 0.02642918, 0.01296473, 0.04587841, 0.03540552,
        0.04089057, 0.03141618, 0.01096952, 0.04089034, 0.00249279,
        0.02044523, 0.02443445, 0.00648224, 0.00648248, 0.01994622,
        0.05934036, 0.        , 0.00448835, 0.0558461 , 0.08724821]),
 'mean_score_time': array([0.02194178, 0.01795208, 0.01894891, 0.01845229, 0.01944864,
        0.02094412, 0.02094424, 0.02293813, 0.02144313, 0.02